In [4]:
# COPOM Statements - Wide Format by Paragraph
# Each column is a meeting (number + date), each row is a paragraph

import pandas as pd
import requests
import html
import re
import time
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# BCB API endpoints
BASE_URL = "https://www.bcb.gov.br/api/servico/sitebcb/copom"
LIST_URL = f"{BASE_URL}/comunicados"
DETAIL_URL = f"{BASE_URL}/comunicados_detalhes"

# Setup session with retry logic
session = requests.Session()
session.headers.update({"User-Agent": "copom-paragraphs/1.0"})
retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

print("Imports and session configured.")

Imports and session configured.


In [5]:
# Fetch list of meetings from BCB API
num_mtg = 30
print(f"Fetching list of last {num_mtg} COPOM meetings...")
response = session.get(LIST_URL, params={"quantidade": num_mtg}, timeout=30)
response.raise_for_status()
meetings_list = response.json()["conteudo"]

# Sort by meeting number (ascending)
meetings_list = sorted(meetings_list, key=lambda x: x['nro_reuniao'])

print(f"Found {len(meetings_list)} meetings")
print(f"Meeting range: {meetings_list[0]['nro_reuniao']} to {meetings_list[-1]['nro_reuniao']}")

Fetching list of last 30 COPOM meetings...
Found 30 meetings
Meeting range: 246 to 275


In [6]:
# Helper functions for HTML parsing

def clean_text_from_html(html_text: str) -> str:
    """Extract clean text from HTML, removing tables."""
    if not html_text:
        return ""
    
    decoded = html.unescape(html_text)
    decoded = re.sub(r'[\u200b\ufeff\u00a0]', '', decoded)
    soup = BeautifulSoup(decoded, "html.parser")
    
    # Remove all tables
    for table in soup.find_all("table"):
        table.decompose()
    
    # Replace <br> with newlines
    for br in soup.find_all("br"):
        br.replace_with("\n")
    
    # Add double newlines after paragraphs
    for p in soup.find_all("p"):
        p.insert_after("\n\n")
    
    text = soup.get_text()
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()


def split_into_paragraphs(text: str) -> list:
    """Split text into a list of paragraphs."""
    if not text:
        return []
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    return paragraphs

print("Helper functions defined.")

Helper functions defined.


In [7]:
# Fetch detailed data for all meetings
meetings_data = {}

print(f"Fetching detailed statements for {len(meetings_list)} meetings...")
for i, meeting in enumerate(meetings_list):
    nro_reuniao = meeting["nro_reuniao"]
    
    try:
        response = session.get(DETAIL_URL, params={"nro_reuniao": nro_reuniao}, timeout=30)
        response.raise_for_status()
        detail = response.json()["conteudo"][0]
        
        # Extract and parse
        html_content = detail.get("textoComunicado", "")
        clean_text = clean_text_from_html(html_content)
        paragraphs = split_into_paragraphs(clean_text)
        
        meetings_data[nro_reuniao] = {
            'date': pd.to_datetime(detail.get("dataReferencia")),
            'titulo': detail.get("titulo"),
            'paragraphs': paragraphs
        }
        
        if (i + 1) % 10 == 0:
            print(f"  Processed {i + 1}/{len(meetings_list)} meetings")
        
        time.sleep(0.15)  # Be respectful to the API
        
    except Exception as e:
        print(f"  Error fetching meeting {nro_reuniao}: {e}")
        continue

print(f"\nSuccessfully retrieved {len(meetings_data)} meetings")

Fetching detailed statements for 30 meetings...
  Processed 10/30 meetings
  Processed 20/30 meetings
  Processed 30/30 meetings

Successfully retrieved 30 meetings


In [8]:
# Create wide-format DataFrame: meetings as columns, paragraphs as rows

# Find max paragraphs across all meetings
max_paragraphs = max(len(m['paragraphs']) for m in meetings_data.values())
print(f"Maximum paragraphs in any meeting: {max_paragraphs}")

# Build the wide dataframe
rows = []
for para_idx in range(max_paragraphs):
    row = {'paragraph': para_idx + 1}
    for nro_reuniao in sorted(meetings_data.keys()):
        meeting = meetings_data[nro_reuniao]
        date_str = meeting['date'].strftime('%Y-%m-%d')
        col_name = f"{nro_reuniao} ({date_str})"
        
        if para_idx < len(meeting['paragraphs']):
            row[col_name] = meeting['paragraphs'][para_idx]
        else:
            row[col_name] = ''
    rows.append(row)

df_wide = pd.DataFrame(rows)
df_wide = df_wide.set_index('paragraph')

print(f"\nCreated wide DataFrame: {len(df_wide)} rows x {len(df_wide.columns)} columns")
print(f"Columns (meetings): {list(df_wide.columns)[:5]} ... {list(df_wide.columns)[-3:]}")
df_wide

Maximum paragraphs in any meeting: 13

Created wide DataFrame: 13 rows x 30 columns
Columns (meetings): ['246 (2022-05-04)', '247 (2022-06-15)', '248 (2022-08-03)', '249 (2022-09-21)', '250 (2022-10-26)'] ... ['273 (2025-09-17)', '274 (2025-11-05)', '275 (2025-12-10)']


,246 (2022-05-04),247 (2022-06-15),248 (2022-08-03),249 (2022-09-21),250 (2022-10-26),251 (2022-12-07),252 (2023-02-01),253 (2023-03-22),254 (2023-05-03),255 (2023-06-21),...,266 (2024-11-06),267 (2024-12-11),268 (2025-01-29),269 (2025-03-19),270 (2025-05-07),271 (2025-06-18),272 (2025-07-30),273 (2025-09-17),274 (2025-11-05),275 (2025-12-10)
paragraph,,,,,,,,,,,,,,,,,,,,,
1,"Em sua 246ª reunião, o Comitê de Política Mone...","Em sua 247ª reunião, o Comitê de Política Mone...","Em sua 248ª reunião, o Comitê de Política Mone...","Em sua 249ª reunião, o Comitê de Política Mone...","Em sua 250ª reunião, o Comitê de Política Mone...","Em sua 251ª reunião, o Comitê de Política Mone...","Em sua 252ª reunião, o Comitê de Política Mone...",Desde a reunião anterior do Comitê de Política...,O ambiente externo se mantém adverso. Os episó...,"O ambiente externo se mantém adverso, ainda qu...",...,"O ambiente externo permanece desafiador, em fu...","O ambiente externo permanece desafiador, em fu...",O ambiente externo permanece desafiador em fun...,O ambiente externo permanece desafiador em fun...,O ambiente externo mostra-se adverso e particu...,O ambiente externo mantém-se adverso e particu...,O ambiente externo está mais adverso e incerto...,O ambiente externo se mantém incerto em função...,O ambiente externo ainda se mantém incerto em ...,O ambiente externo ainda se mantém incerto em ...
2,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,A atualização do cenário do Copom pode ser des...,"Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...",...,"Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do...","Em relação ao cenário doméstico, o conjunto do..."
3,O ambiente externo seguiu se deteriorando. As ...,"O ambiente externo seguiu se deteriorando, mar...",O ambiente externo mantém-se adverso e volátil...,O ambiente externo mantém-se adverso e volátil...,O ambiente externo mantém-se adverso e volátil...,O ambiente externo mantém-se adverso e volátil...,O ambiente externo segue marcado pela perspect...,"Na mesma linha, as projeções de inflação do Co...",As projeções de inflação do Copom em seu cenár...,As projeções de inflação do Copom em seu cenár...,...,As expectativas de inflação para 2024 e 2025 a...,As expectativas de inflação para 2024 e 2025 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...,As expectativas de inflação para 2025 e 2026 a...
4,"Considerando os cenários avaliados, o balanço ...","Considerando os cenários avaliados, o balanço ...",Notou-se que as projeções de inflação para os ...,"Considerando os cenários avaliados, o balanço ...","Considerando os cenários avaliados, o balanço ...","Considerando os cenários avaliados, o balanço ...","A conjuntura, particularmente incerta no âmbit...","O Comitê ressalta que, em seus cenários para a...","O Comitê ressalta que, em seus cenários para a...","O Comitê ressalta que, em seus cenários para a...",...,O Comitê avalia que há uma assim

In [9]:
# Summary statistics
print("COPOM Paragraphs Dataset Summary")
print("=" * 50)
print(f"Total meetings: {len(df_wide.columns)}")
print(f"Total paragraph rows: {len(df_wide)}")

# Count non-empty paragraphs per meeting
para_counts = (df_wide != '').sum()
print(f"\nParagraphs per meeting:")
print(f"  Min: {para_counts.min()}")
print(f"  Max: {para_counts.max()}")
print(f"  Mean: {para_counts.mean():.1f}")

# Show paragraph counts by meeting
print(f"\nParagraph counts by meeting:")
para_counts

COPOM Paragraphs Dataset Summary
Total meetings: 30
Total paragraph rows: 13

Paragraphs per meeting:
  Min: 7
  Max: 13
  Mean: 10.5

Paragraph counts by meeting:


246 (2022-05-04)     8
247 (2022-06-15)     8
248 (2022-08-03)     9
249 (2022-09-21)     7
250 (2022-10-26)     7
251 (2022-12-07)     7
252 (2023-02-01)     8
253 (2023-03-22)     9
254 (2023-05-03)     9
255 (2023-06-21)     8
256 (2023-08-02)    10
257 (2023-09-20)    10
258 (2023-11-01)    11
259 (2023-12-13)    11
260 (2024-01-31)    11
261 (2024-03-20)    12
262 (2024-05-08)    11
263 (2024-06-19)    11
264 (2024-07-31)    12
265 (2024-09-18)    12
266 (2024-11-06)    12
267 (2024-12-11)    13
268 (2025-01-29)    13
269 (2025-03-19)    13
270 (2025-05-07)    13
271 (2025-06-18)    12
272 (2025-07-30)    12
273 (2025-09-17)    12
274 (2025-11-05)    12
275 (2025-12-10)    12
dtype: int64

In [10]:
# Helper to view a specific paragraph across meetings

def compare_paragraph(para_num: int, last_n: int = 5):
    """Compare a specific paragraph across the last N meetings."""
    if para_num not in df_wide.index:
        print(f"Paragraph {para_num} not found (max: {df_wide.index.max()})")
        return
    
    cols = df_wide.columns[-last_n:]
    print(f"Paragraph {para_num} across last {last_n} meetings:")
    print("=" * 70)
    
    for col in cols:
        text = df_wide.loc[para_num, col]
        if text:
            print(f"\n[{col}]")
            print(f"{text[:300]}..." if len(text) > 300 else text)
        else:
            print(f"\n[{col}] (no paragraph {para_num})")

# Example: Compare first paragraph across last 5 meetings
compare_paragraph(1, last_n=5)

Paragraph 1 across last 5 meetings:

[271 (2025-06-18)]
O ambiente externo mantém-se adverso e particularmente incerto em função da conjuntura e da política econômica nos Estados Unidos, principalmente acerca de suas políticas comercial e fiscal e de seus respectivos efeitos. Além disso, o comportamento e a volatilidade de diferentes classes de ativos ta...

[272 (2025-07-30)]
O ambiente externo está mais adverso e incerto em função da conjuntura e da política econômica nos Estados Unidos, principalmente acerca de suas políticas comercial e fiscal e de seus respectivos efeitos. Consequentemente, o comportamento e a volatilidade de diferentes classes de ativos têm sido afe...

[273 (2025-09-17)]
O ambiente externo se mantém incerto em função da conjuntura e da política econômica nos Estados Unidos. Consequentemente, o comportamento e a volatilidade de diferentes classes de ativos têm sido afetados, com reflexos nas condições financeiras globais. Tal cenário exige particular cautela por 

In [11]:
# Export to clipboard or CSV
df_wide.to_clipboard()
print("DataFrame copied to clipboard!")

# Optionally save to CSV
# df_wide.to_csv("copom_paragraphs_wide.csv")
# print("Saved to copom_paragraphs_wide.csv")

DataFrame copied to clipboard!
